<a href="https://colab.research.google.com/github/khaledAIVR/Rooaa/blob/master/ROOAA_Notebook_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *YOLO object detection*

In [ ]:
cd /content

In [ ]:
!git clone https://github.com/pjreddie/darknet

In [ ]:
cd darknet

In [ ]:
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make
!wget https://pjreddie.com/media/files/yolov3.weights
!pip install pydub
!pip install opencv-python
!pip install gtts

In [ ]:
#yolo.py
import os
import glob
import time
import cv2 
import numpy as np
import imutils
import subprocess

def detect_and_locate_obj(path, LABELS, net, ln):
  
  # initialize
  # load the image with imread()
  imageSource = path
  img = cv2.imread(imageSource)


  COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
    dtype="uint8")


  # grab the image dimensions and convert it to a blob
  (H, W) = img.shape[:2]
  # construct a blob from the input image and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes and
  # associated probabilities
  blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
  swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)


  # initialize our lists of detected bounding boxes, confidences, and
  # class IDs, respectively
  boxes = []
  confidences = []
  classIDs = []
  detected = []
  centers = []

  # loop over each of the layer outputs
  for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
        # extract the class ID and confidence (i.e., probability) of
        # the current object detection
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]


        # filter out weak predictions by ensuring the detected
        # probability is greater than the minimum probability
        if confidence > 0.5:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")


            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))


            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            centers.append((centerX, centerY))

              
  # apply non-maxima suppression to suppress weak, overlapping bounding
  # boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

  classes = []
  locations = []
  center_axis = []
  # ensure at least one detection exists
  if len(idxs) > 0: 
    # loop over the indexes we are keeping
    for i in idxs.flatten():
      # find  
      centerX, centerY = centers[i][0], centers[i][1]


      #-------------------------Locate objects-------------------------"""
      if centerX <= W/3:
        location = "left "
      elif centerX <= (W/3 * 2):
        location = "center "
      else:
        location = "right "

      locations.append(location)
      classes.append(LABELS[classIDs[i]])
      center_axis.append([centerX, centerY])



  return classes, locations, center_axis

# The Monocular Depth Estimation

In [ ]:
cd /content

In [ ]:
!git clone https://github.com/ialhashim/DenseDepth.git
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

In [ ]:
#layers.py
from keras.engine.topology import Layer, InputSpec
import keras.utils.conv_utils as conv_utils
import tensorflow as tf
import keras.backend as K

class BilinearUpSampling2D(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(BilinearUpSampling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
            return (input_shape[0],
                    input_shape[1],
                    height,
                    width)
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
            return (input_shape[0],
                    height,
                    width,
                    input_shape[3])

    def call(self, inputs):
        input_shape = K.shape(inputs)
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
        
        return tf.image.resize_images(inputs, [height, width], method=tf.image.ResizeMethod.BILINEAR, align_corners=True)

    def get_config(self):
        config = {'size': self.size, 'data_format': self.data_format}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


# Import libs, define functions and load the two models

In [ ]:
import os
import glob
import time
import cv2 
import numpy as np
import imutils
import subprocess
import random    
import socket
import threading
import IPython
import portpicker

from keras.models import load_model
from matplotlib import pyplot as plt
from six.moves import SimpleHTTPServer
from six.moves import socketserver
from google.colab import output
from google.colab import files
from IPython.display import Audio
from IPython.display import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from gtts import gTTS
#------------------------------------------------------



"""/////////////////////////////////////////////////"""
#danger_dictionary takes the label index as a key and the value is the estimated danger value
danger_dict = {0: 9, 1: 8, 2: 10, 3: 9, 5: 10, 7: 10, 9: 10, 11: 10, 12: 8, 13: 6, 14: 4, 15: 4, 16: 5, 31: 5, 32: 7, 34: 7, 36: 7, 56: 8, 57: 8, 58: 8, 59: 8, 60: 8, 61: 8, 62: 5, 67: 5, 68: 7, 69: 7, 71: 7, 72: 6, 75: 5, 4: 0, 6: 0, 8: 0, 10: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 33: 0, 35: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 63: 0, 64: 0, 65: 0, 66: 0, 70: 0, 73: 0, 74: 0, 76: 0, 77: 0, 78: 0, 79: 0} 
labels_of_coco = ["person","bicycle","car","motorbike",'aeroplane',"bus","train","truck","boat","traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork","knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake","chair","sofa","pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"]

class filtration: 
    label = -1 
    location = -1
    depth = -1
    def __init__(self, label, location, depth):
       self.label = label
       self.location = location
       self.depth = depth

def most_dangerous(arr):
  if len(arr) > 0:
    max_obj = arr[0]
    for i in arr:
      if danger_dict[labels_of_coco.index(i.label)] >= danger_dict[labels_of_coco.index(max_obj.label)]:
        max_obj = i
    return max_obj
  return -1         
#------------------------------------------------------       



"""/////////////////////////////////////////////////"""
def resize(path):
  """Takes a jpg, resize it to 480*640 png image"""
  imageFile = path
  img = cv2.imread(imageFile)
  width = 480
  height = 640
  dim = (width, height)
  img = cv2.resize(img, dim)   
  cv2.imwrite(path, img)
#------------------------------------------------------  



"""/////////////////////////////////////////////////"""
def depth_approx(path, centers):
  img = cv2.imread(path)
  depthes = [] 
  for axis in centers:
    depth = img[axis[1], axis[0]][0]
    depthes.append(depth)
  return depthes 
#------------------------------------------------------



"""/////////////////////////////////////////////////"""
#Using the capture snippet and modify it to take a capture automatically
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename  
#------------------------------------------------------



"""/////////////////////////////////////////////////"""
#Used to run the Colab-Audio service in a loop
class _V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class _FileHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  """SimpleHTTPRequestHandler with a couple tweaks."""

  def translate_path(self, path):
    # Client specifies absolute paths.
    return path

  def log_message(self, fmt, *args):
    # Suppress logging since it's on the background. Any errors will be reported
    # via the handler.
    pass

  def end_headers(self):
    # Do not cache the response in the notebook, since it may be quite large.
    self.send_header('x-colab-notebook-cache-control', 'no-cache')
    SimpleHTTPServer.SimpleHTTPRequestHandler.end_headers(self)


def play_audio(filename):
  """Downloads the file to the user's local disk via a browser download action.

  Args:
    filename: Name of the file on disk to be downloaded.
  """

  started = threading.Event()
  port = portpicker.pick_unused_port()

  def server_entry():
    httpd = _V6Server(('::', port), _FileHandler)
    started.set()
    # Serve multiple requests, in case the audio is played more than once.
    httpd.serve_forever()

  thread = threading.Thread(target=server_entry)
  thread.start()
  started.wait()

  output.eval_js("""
    (()=> {
      const audio = document.createElement('audio');
      audio.controls = true;
      audio.autoplay = true;
      audio.src = `https://localhost:%(port)d%(path)s`;
      document.body.appendChild(audio);
    })()
  """% {'port': port, 'path': os.path.abspath(filename)})
#------------------------------------------------------



"""Load the MDE model"""
# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')
# Load model into GPU / CPU
model = load_model("/content/DenseDepth/nyu.h5", custom_objects=custom_objects, compile=False)
print('\nModel loaded ({0}).'.format("/content/DenseDepth/nyu.h5"))
#------------------------------------------------------



"""Load YOLO model"""
# load the COCO class labels our YOLO model was trained on
LABELS = open("/content/darknet/data/coco.names").read().strip().split("\n")

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet("/content/darknet/cfg/yolov3.cfg", "/content/darknet/yolov3.weights")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#------------------------------------------------------

In [ ]:
#utils.py
import numpy as np
from PIL import Image

def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale 
    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    predictions = model.predict(images, batch_size=batch_size)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=1000), minDepth, maxDepth) / maxDepth

def scale_up(scale, images):
    from skimage.transform import resize
    scaled = []
    
    for i in range(len(images)):
        img = images[i]
        output_shape = (scale * img.shape[0], scale * img.shape[1])
        scaled.append( resize(img, output_shape, order=1, preserve_range=True, mode='reflect', anti_aliasing=True ) )

    return np.stack(scaled)

def load_images(image_files):
    loaded_images = []
    for file in image_files:
        x = np.clip(np.asarray(Image.open( file ), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def to_multichannel(i):
    if i.shape[2] == 3: return i
    i = i[:,:,0]
    return np.stack((i,i,i), axis=2)
        
def display_images(outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize

    plasma = plt.get_cmap('plasma')

    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    
    all_images = []

    for i in range(outputs.shape[0]):
        imgs = []
        
        if isinstance(inputs, (list, tuple, np.ndarray)):
            x = to_multichannel(inputs[i])
            x = resize(x, shape, preserve_range=True, mode='reflect', anti_aliasing=True )
            imgs.append(x)

        if isinstance(gt, (list, tuple, np.ndarray)):
            x = to_multichannel(gt[i])
            x = resize(x, shape, preserve_range=True, mode='reflect', anti_aliasing=True )
            imgs.append(x)

        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
            imgs.append(plasma(rescaled)[:,:,:3])
        else:
            imgs.append(to_multichannel(outputs[i]))

        img_set = np.hstack(imgs)
        all_images.append(img_set)

    all_images = np.stack(all_images)
    
    return skimage.util.montage(all_images, multichannel=True, fill=(0,0,0))

def save_images(filename, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=False):
    montage =  display_images(outputs, inputs, is_colormap, is_rescale)
    im = Image.fromarray(np.uint8(montage*255))
    im.save(filename)

def load_test_data(test_data_zip_file='nyu_test.zip'):
    print('Loading test data...', end='')
    import numpy as np
    from data import extract_zip
    data = extract_zip(test_data_zip_file)
    from io import BytesIO
    rgb = np.load(BytesIO(data['eigen_test_rgb.npy']))
    depth = np.load(BytesIO(data['eigen_test_depth.npy']))
    crop = np.load(BytesIO(data['eigen_test_crop.npy']))
    print('Test data loaded.\n')
    return {'rgb':rgb, 'depth':depth, 'crop':crop}

def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()
    abs_rel = np.mean(np.abs(gt - pred) / gt)
    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())
    log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()
    return a1, a2, a3, abs_rel, rmse, log_10

def evaluate(model, rgb, depth, crop, batch_size=6, verbose=False):
    N = len(rgb)

    bs = batch_size

    predictions = []
    testSetDepths = []
    
    for i in range(N//bs):    
        x = rgb[(i)*bs:(i+1)*bs,:,:,:]
        
        # Compute results
        true_y = depth[(i)*bs:(i+1)*bs,:,:]
        pred_y = scale_up(2, predict(model, x/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0
        
        # Test time augmentation: mirror image estimate
        pred_y_flip = scale_up(2, predict(model, x[...,::-1,:]/255, minDepth=10, maxDepth=1000, batch_size=bs)[:,:,:,0]) * 10.0

        # Crop based on Eigen et al. crop
        true_y = true_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y = pred_y[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        pred_y_flip = pred_y_flip[:,crop[0]:crop[1]+1, crop[2]:crop[3]+1]
        
        # Compute errors per image in batch
        for j in range(len(true_y)):
            predictions.append(   (0.5 * pred_y[j]) + (0.5 * np.fliplr(pred_y_flip[j]))   )
            testSetDepths.append(   true_y[j]   )

    predictions = np.stack(predictions, axis=0)
    testSetDepths = np.stack(testSetDepths, axis=0)

    e = compute_errors(predictions, testSetDepths)

    if verbose:
        print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
        print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))

    return e

# ROOAA App

In [ ]:
while True: 
  """-------------------------Take a picture-------------------------"""
  try:
    filename = take_photo()
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))



  """-------------------------YOLO obj detection and location-------------------------"""
  resize("/content/photo.jpg")
  classes, locations, centers = detect_and_locate_obj('/content/photo.jpg', LABELS, net, ln)



  """-------------------------Estimate the distance-------------------------"""
  inputs = load_images( glob.glob("/content/photo.jpg") )

  outputs = predict(model, inputs)

  save_images('/content/output.jpg', outputs)

  resize('/content/output.jpg')

  depthes = depth_approx("/content/output.jpg", centers)



  """-------------------------filtration-------------------------"""
  filtration_arr = []
  warning_area = []
  informing_area = []
  left_objs = []
  right_objs = []
  center_objs = []
  texts = ""
  #---------------------------------
  for i in range(len(classes)):
    obj = filtration(classes[i], locations[i], depthes[i])
    filtration_arr.append(obj)
  #---------------------------------
  for i in filtration_arr:
    if i.depth <= 35:
      warning_area.append(i)   
    else:
      informing_area.append(i)   
  #---------------------------------
  if  len(warning_area) > 0:
    for i in warning_area:
      if i.location == 'center ':
        center_objs.append(i)
      elif i.location =='left ':  
        left_objs.append(i)
      elif i.location == 'right ':  
        right_objs.append(i)

    most_dangerous_center = most_dangerous(center_objs)
    most_dangerous_left = most_dangerous(left_objs)
    most_dangerous_right = most_dangerous(right_objs)

    texts =  "Watch out "
    if most_dangerous_center != -1  :
      texts += most_dangerous_center.location + " " + most_dangerous_center.label + " "

    if most_dangerous_left != -1:
      texts += most_dangerous_left.location + " " + most_dangerous_left.label + " "
   
    if most_dangerous_right != -1:
      texts += most_dangerous_right.location + " " + most_dangerous_right.label + " "
  #---------------------------------
  elif len(informing_area) > 0:   
    for i in informing_area:
      if i.location == 'center ':
        center_objs.append(i)
      elif i.location =='left ':  
        left_objs.append(i)
      elif i.location == 'right ':  
        right_objs.append(i)

    most_dangerous_center = most_dangerous(center_objs)
    most_dangerous_left = most_dangerous(left_objs)
    most_dangerous_right = most_dangerous(right_objs)

    texts =  " "
    if most_dangerous_center != -1  :
      texts += most_dangerous_center.location + " " + most_dangerous_center.label + " "

    if most_dangerous_left != -1:
      texts += most_dangerous_left.location + " " + most_dangerous_left.label + " "
   
    if most_dangerous_right != -1:
      texts += most_dangerous_right.location + " " + most_dangerous_right.label + " "
  #---------------------------------

  

  """-------------------------Convert text to speech-------------------------"""
  if not texts:
    texts = "Nothing"

  tts = gTTS(text=texts, lang='en')

  tts.save("ttsFile.mp3")  



  """-------------------------Play Audio-------------------------"""
  play_audio("/content/ttsFile.mp3")
  time.sleep(1)